In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# Link data from Google Drive(full path)
link_parent_path = "/content/drive/MyDrive/Training Lab/Phase 1: Basic ML/Session 1"
link_data = link_parent_path + "/dataset/data_RidgeRegression.txt"

In [ ]:
# Early preprocess raw data
data = list()
with open(link_data, "r") as f:
  raw_data = f.read().split("\n")
  for i in range(len(raw_data)):
    list_data = list(raw_data[i].split(" "))
    list_data = [float(list_data[j]) for j in range(len(list_data)) if list_data[j] != '']
    data.append(list_data)

np_data = np.array(data)
np_data = np_data[:,1:]
print(np_data)

In [ ]:
num_examples = len(data)
num_features = len(data[0])

print(num_examples)
print(num_features)

60
17


In [ ]:
# Normalize data(feature scaling)

def min_max_scaling(vect):
  x_max = np.max(vect)
  x_min = np.min(vect)
  normalized_vect = (vect - x_min) / (x_max - x_min)
  return normalized_vect

for col in range(0, num_features-1):
  np_data[:, col] = min_max_scaling(np_data[:, col])

print(np_data)

In [ ]:
# Add interception to Array
npdata = np.hstack((np.ones((num_examples, 1)),np_data))
print(npdata)

In [ ]:
# Add columns' names for pandas dataframe
col_names = list()
for i in range(0, num_features - 1):
  name = str()
  name = "A" + str(i)
  col_names.append(name)

col_names.append("Death rates")
pd_data = pd.DataFrame(npdata, columns = col_names)
pd_data.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,Death rates
0,1.0,0.52,0.211268,0.363636,0.403226,0.688525,0.727273,0.615063,0.218213,0.212202,0.339768,0.135294,0.030912,0.044025,0.209386,0.600000,0.406723
1,1.0,0.50,0.154930,0.409091,0.887097,0.360656,0.606061,0.502092,0.343909,0.074271,0.652510,0.294118,0.010819,0.028302,0.137184,0.542857,0.642454
2,1.0,0.68,0.239437,0.500000,0.774194,0.475410,0.242424,0.619247,0.341366,0.000000,0.216216,0.176471,0.007728,0.015723,0.115523,0.457143,0.532285
3,1.0,0.74,0.464789,0.727273,0.145161,0.803279,0.636364,0.447699,0.203923,0.697613,0.633205,0.658824,0.026275,0.022013,0.083032,0.514286,0.594120
4,1.0,0.66,0.323944,0.636364,0.322581,0.852459,0.181818,0.744770,0.605473,0.625995,0.382239,0.288235,0.064915,0.116352,0.740072,0.485714,0.870149


In [ ]:
# Build Ridge Regression model

class RidgeRegression():

  def __init__(self, theta = None, lamb = None, lr = None):
    self.theta = theta
    self.lamb = lamb
    self.learning_rate = lr

  def fit(self, x_train, y_train, lamb):
    self.lamb = lamb
    theta = np.linalg.inv(np.transpose(x_train).dot(x_train) + self.lamb * np.eye(num_features - 1)).dot(np.transpose(x_train)).dot(y_train)
    self.theta = theta
    return theta

  def data_generator(self, x_train, y_train, batch_size, randomize = False):
    num_examples = len(x_train)
    num_features = len(x_train[0])
    if randomize == True:
      ind_arr = np.arange(num_examples)
      np.random.shuffle(ind_arr)
      x_train,y_train = x_train[ind_arr],y_train[ind_arr]
    for i in range(0, num_examples, batch_size):
      idx = i
      idy = min(i+batch_size, num_examples)
      yield x_train[idx:idy], y_train[idx:idy]

  def loss_function(self, x_train, y_train, theta):
    num_examples = len(x_train)
    num_features = len(x_train[0])
    theta = np.reshape(theta, (-1,1))
    return (1/num_examples) * np.transpose(y_train - x_train.dot(theta)).dot(y_train - x_train.dot(theta))

  def rss(self, y_new, y_predict):
    return np.sum((y_new - y_predict)**2) / (y_new.shape[0])

  def fit_gradient(self, x_train, y_train, lamb, batch_size, lr, maximum_epochs = 1000):
    num_examples = len(x_train)
    num_features = len(x_train[0])
    self.lamb = lamb
    theta = np.random.randn(num_features,1)
    self.learning_rate = lr
    i = 0
    while True:
      i += 1
      if i == maximum_epochs:
        break
      if i % 100 == 0:
        print(f"Iteration: {i}, Loss function: {self.loss_function(x_train, y_train, theta)[0][0]}")
      
      prev_theta = theta
      batch_obj = self.data_generator(x_train, y_train, batch_size, True)
      for x_batch, y_batch in batch_obj:
        grad_x = np.transpose(x_batch).dot(x_batch.dot(theta) - y_batch) + lamb * theta
        theta = theta - lr * grad_x
        if np.linalg.norm(theta - prev_theta) < 1e-5:
          print("Terminal")
          break
    self.theta = theta
    return theta

  def predict(self, x_test, theta):
    x_test = np.array(x_test)
    return x_test.dot(theta)

  def get_the_best_lambda(self, X_train, y_train):

    def cross_validation(num_folds, lamb):
      row_ids = np.array(range(X_train.shape[0]))
      valid_ids = np.split(row_ids[:len(row_ids) - len(row_ids) % num_folds], num_folds)
      valid_ids[-1] = np.append(valid_ids[-1], row_ids[len(row_ids) - len(row_ids) % num_folds:])
      train_ids = [[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
      aver_RSS = 0
      for i in range(num_folds):
          valid_part = {'X': X_train[valid_ids[i]], 'y': y_train[valid_ids[i]]}
          train_part = {'X': X_train[train_ids[i]], 'y': y_train[train_ids[i]]}
          theta = self.fit(train_part['X'], train_part['y'], lamb)
          y_predicted = self.predict(valid_part['X'], theta)
          aver_RSS += self.rss(valid_part['X'], valid_part['y'])
      return aver_RSS / num_folds

    def range_scan(best_lambda, minimum_RSS, lambda_values):
      for lamb in lambda_values:
          aver_RSS = cross_validation(5, lamb)
      if aver_RSS < minimum_RSS:
          best_LAMBDA = lamb
          minimum_RSS = aver_RSS
      return best_lambda, minimum_RSS

    best_lambda, minimum_RSS = range_scan(0, 10000 ** 2, range(X_train.shape[0]))
    # Search for local range of best_lambda. Make sure that lambda_value > 0 by setting max()
    lambda_values = [k * 1/1000 for k in range(max(0, (best_lambda - 1) * 1000), (best_lambda + 1) * 1000)] 
    best_lambda, minimum_RSS = range_scan(best_lambda, minimum_RSS, lambda_values)
    self.lamb = best_lambda
    return best_lambda


In [ ]:
# Split training and test sets

num_training_set = 50
x_train = npdata[:, :num_features-1]
y_train = npdata[:, -1]

X_train = x_train[:num_training_set]
Y_train = y_train[:num_training_set].reshape(-1,1)
X_test = x_train[num_training_set:]
Y_test = y_train[num_training_set:].reshape(-1,1)

In [ ]:
rr = RidgeRegression()
best_lambda = rr.get_the_best_lambda(X_train, Y_train)

theta = rr.fit(X_train, Y_train, best_lambda)
y_predicted = rr.predict(X_test, theta)

rr.rss(Y_test, y_predicted)

0.015242347859489632

# Scikit-learn

In [ ]:
from sklearn.linear_model import RidgeCV   
from sklearn.metrics import mean_squared_error 

# GridSearch
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])

clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_test)
print('RSS: ', mean_squared_error(Y_test, y_predicted))

RSS:  0.015242347859489632
